In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
# create spark sessiom
spark=SparkSession.builder.getOrCreate()

In [6]:
sdf=spark.read.csv('Bank Churn Modelling.csv',header=True,inferSchema=True)

In [7]:
sdf.show()

+----------+---------+-----------+---------+------+---+------+---------+---------------+---------------+----------------+----------------+-----+
|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|Num Of Products|Has Credit Card|Is Active Member|Estimated Salary|Churn|
+----------+---------+-----------+---------+------+---+------+---------+---------------+---------------+----------------+----------------+-----+
|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|              1|              1|               1|       101348.88|    1|
|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|              1|              0|               1|       112542.58|    0|
|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|              3|              1|               0|       113931.57|    1|
|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|              2|              0|               0|        9

In [8]:
sdf.columns

['CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'Num Of Products',
 'Has Credit Card',
 'Is Active Member',
 'Estimated Salary',
 'Churn']

In [9]:
sdf=sdf.dropna()

In [10]:
sdf.printSchema

<bound method DataFrame.printSchema of DataFrame[CustomerId: int, Surname: string, CreditScore: int, Geography: string, Gender: string, Age: int, Tenure: int, Balance: double, Num Of Products: int, Has Credit Card: int, Is Active Member: int, Estimated Salary: double, Churn: int]>

In [11]:
sdf.show()

+----------+---------+-----------+---------+------+---+------+---------+---------------+---------------+----------------+----------------+-----+
|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|Num Of Products|Has Credit Card|Is Active Member|Estimated Salary|Churn|
+----------+---------+-----------+---------+------+---+------+---------+---------------+---------------+----------------+----------------+-----+
|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|              1|              1|               1|       101348.88|    1|
|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|              1|              0|               1|       112542.58|    0|
|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|              3|              1|               0|       113931.57|    1|
|  15701354|     Boni|        699|   France|Female| 39|     1|      0.0|              2|              0|               0|        9

In [12]:
sdf.dtypes

[('CustomerId', 'int'),
 ('Surname', 'string'),
 ('CreditScore', 'int'),
 ('Geography', 'string'),
 ('Gender', 'string'),
 ('Age', 'int'),
 ('Tenure', 'int'),
 ('Balance', 'double'),
 ('Num Of Products', 'int'),
 ('Has Credit Card', 'int'),
 ('Is Active Member', 'int'),
 ('Estimated Salary', 'double'),
 ('Churn', 'int')]

In [13]:
sdf.columns

['CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'Num Of Products',
 'Has Credit Card',
 'Is Active Member',
 'Estimated Salary',
 'Churn']

In [14]:
from pyspark.ml.feature import StringIndexer

In [15]:
si=StringIndexer(inputCols=['Geography','Gender'],outputCols=['iGeography','iGender'])

In [16]:
sdf=si.fit(sdf).transform(sdf)

In [17]:
sdf.columns

['CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'Num Of Products',
 'Has Credit Card',
 'Is Active Member',
 'Estimated Salary',
 'Churn',
 'iGeography',
 'iGender']

In [18]:
from pyspark.ml.feature import VectorAssembler

In [19]:
va=VectorAssembler(inputCols=['CreditScore',
'Tenure',
 'Balance',
 'Num Of Products',
 'Has Credit Card',
 'Is Active Member',
 'Estimated Salary',
 'iGeography',
 'iGender'],outputCol='X')

In [20]:
sdf=va.transform(sdf)

In [21]:
sdf.show()

+----------+---------+-----------+---------+------+---+------+---------+---------------+---------------+----------------+----------------+-----+----------+-------+--------------------+
|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|Num Of Products|Has Credit Card|Is Active Member|Estimated Salary|Churn|iGeography|iGender|                   X|
+----------+---------+-----------+---------+------+---+------+---------+---------------+---------------+----------------+----------------+-----+----------+-------+--------------------+
|  15634602| Hargrave|        619|   France|Female| 42|     2|      0.0|              1|              1|               1|       101348.88|    1|       0.0|    1.0|[619.0,2.0,0.0,1....|
|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|              1|              0|               1|       112542.58|    0|       2.0|    1.0|[608.0,1.0,83807....|
|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|   

In [22]:
data=sdf.select(['Churn','X'])

In [23]:
data.show()

+-----+--------------------+
|Churn|                   X|
+-----+--------------------+
|    1|[619.0,2.0,0.0,1....|
|    0|[608.0,1.0,83807....|
|    1|[502.0,8.0,159660...|
|    0|[699.0,1.0,0.0,2....|
|    0|[850.0,2.0,125510...|
|    1|[645.0,8.0,113755...|
|    0|[822.0,7.0,0.0,2....|
|    1|[376.0,4.0,115046...|
|    0|[501.0,4.0,142051...|
|    0|[684.0,2.0,134603...|
|    0|[528.0,6.0,102016...|
|    0|[497.0,3.0,0.0,2....|
|    0|[476.0,10.0,0.0,2...|
|    0|[549.0,5.0,0.0,2....|
|    0|[635.0,7.0,0.0,2....|
|    0|[616.0,3.0,143129...|
|    1|[653.0,1.0,132602...|
|    0|[549.0,9.0,0.0,2....|
|    0|[587.0,6.0,0.0,1....|
|    0|[726.0,6.0,0.0,2....|
+-----+--------------------+
only showing top 20 rows



In [24]:
train, test= data.randomSplit([0.7,0.3],seed=2529)

In [25]:
train.columns

['Churn', 'X']

In [26]:
from pyspark.ml.feature import StandardScaler

In [27]:
ss=StandardScaler(inputCol='X',outputCol="sX",withStd=True,withMean=True)

In [28]:
strain=ss.fit(train).transform(train)

In [29]:
stest=ss.fit(train).transform(test)

In [30]:
from pyspark.ml.classification import LogisticRegression

In [31]:
logit=LogisticRegression(featuresCol='sX',labelCol='Churn')

In [32]:
results=logit.fit(strain).transform(stest)

In [34]:
logit.fit(strain)

LogisticRegressionModel: uid=LogisticRegression_42228460d712, numClasses=2, numFeatures=9

In [36]:
results=logit.fit(strain).transform(stest)

In [37]:
results

DataFrame[Churn: int, X: vector, sX: vector, rawPrediction: vector, probability: vector, prediction: double]

In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [39]:
mce= MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Churn',metricName='accuracy')

In [40]:
mce.evaluate(results)

0.8056300268096515

In [41]:
from pyspark.ml.classification import RandomForestClassifier

In [42]:
rfc= RandomForestClassifier(featuresCol='X',labelCol='Churn')

In [43]:
results=rfc.fit(train).transform(test)

In [44]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [45]:
mce= MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='Churn',metricName='accuracy')

In [46]:
mce.evaluate(results)

0.8260723860589813

In [55]:
from pyspark.ml.classification import LogisticRegression

In [59]:
lr=LogisticRegression(featuresCol='X',labelCol='Churn')

In [60]:
results=lr.fit(train).transform(test)

In [61]:
lr.fit(train).transform(test).show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|                   X|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(9,[0,1,3,6],[413...|[1.82605480013236...|[0.86129107014923...|       0.0|
|    0|(9,[0,1,3,6],[474...|[1.70242670490369...|[0.84585141083148...|       0.0|
|    0|(9,[0,1,3,6],[497...|[1.72757032530748...|[0.84910137431828...|       0.0|
|    0|(9,[0,1,3,6],[501...|[1.78514231311908...|[0.85633067669134...|       0.0|
|    0|(9,[0,1,3,6],[532...|[1.81798220447741...|[0.86032383138104...|       0.0|
|    0|(9,[0,1,3,6],[537...|[1.67210573050045...|[0.84185636780065...|       0.0|
|    0|(9,[0,1,3,6],[544...|[1.71501257302707...|[0.84748530825935...|       0.0|
|    0|(9,[0,1,3,6],[552...|[1.72560146454876...|[0.84884893424467...|       0.0|
|    0|(9,[0,1,3,6],[554...|[1.68358715301369...|[0.84337894523666...|       0.0|
|    0|(9,[0,1,3

In [63]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [66]:
mce=MulticlassClassificationEvaluator(labelCol='Churn',predictionCol='prediction',metricName='accuracy')

mce.evaluate(results)

In [67]:
mce.evaluate(results)

0.8056300268096515